# Clase 1 — Fundamentos de Visión por Computador

---



## Resumen

* Conceptos básicos de imágenes digitales y formatos de archivo.
* Uso práctico de **OpenCV** en Python para carga, manipulación y análisis básico de imágenes.
* Ejercicios guiados paso a paso con código limpio, tipado y pruebas básicas.



---

## Objetivos de aprendizaje

Al finalizar la sesión, los estudiantes serán capaces de:

1. Explicar qué es una imagen digital, cómo se representa en memoria y cuáles son las diferencias entre formatos comunes (PNG, JPEG, TIFF, BMP, WEBP).
2. Manipular imágenes con OpenCV: carga, guardado, cambio de tamaño, recorte, rotación y conversión de espacios de color.
3. Realizar análisis básico de imágenes: histogramas, equalización, detección de bordes y segmentación simple por color.
4. Escribir código Python legible, tipado y probado que implemente pipelines simples de visión por computador.

---



## Requisitos previos y recursos

Requisitos técnicos:

* Python 3.9+ (recomendado 3.10 o 3.11).
* Espacio en disco para imágenes de ejemplo (100 MB es más que suficiente para prácticas).
* Acceso a JupyterLab o un IDE (VS Code recomendado).

Paquetes que instalaremos (se detalla `requirements.txt` más abajo):

* `opencv-python` (OpenCV - para carga, manipulación y análisis básico de imágenes)
* `numpy` (ETL + Analisis estadístico de dataset)
* `matplotlib` (para visualización en notebooks)
* `pytest` (para pruebas)
* `mypy` (opcional, para chequeo de tipos)



---

Contenido sugerido para `requirements.txt`:

```
opencv-python>=4.5
numpy>=1.23
matplotlib>=3.5
pytest>=7.0
mypy>=0.990
```

---



In [ ]:
# Instalación de dependencias:

# Crear requirements.txt
with open("requirements.txt", "w") as f:
    f.write("""opencv-python>=4.5
numpy>=1.23
matplotlib>=3.5
pytest>=7.0
mypy>=0.990
""")

# Instalar desde requirements.txt
!pip install -r requirements.txt



## Parte Teórica (detallada)

### 1) ¿Qué es una imagen digital?

* Una imagen digital se representa como una **matriz** (array) de valores discretos. Cada posición de la matriz es un *pixel* (picture element).
* Cada píxel puede tener uno o varios canales (por ejemplo, escala de grises — 1 canal; RGB — 3 canales). En OpenCV las imágenes con color se cargan por defecto como **BGR** (Azul, Verde y Rojo).
* **Muestreo** (sampling): número de píxeles por unidad de espacio; resolución.
* **Cuantización** (quantization): niveles discretos por canal, típicamente 8 bits por canal (0–255). También existen imágenes de 16 bits o floating point para procesamiento científico.

**Representación en memoria**: tipicamente `np.ndarray` con shape `(alto, ancho)` para escala de grises o `(alto, ancho, canales)` para color. dtype común: `uint8`, `uint16`, `float32`.

### 2) Espacios de color

* **RGB**: esquema aditivo más común para cámaras y pantallas.
* **BGR**: convención usada por OpenCV (tener cuidado al mostrar con `matplotlib`, que espera RGB).
* **HSV / HSL**: HSV (Tono, Saturación y Valor) y HSL (Tono, Saturación y Luminosidad). Útil para segmentación por color (separa tono del brillo/ saturación).
* **YCbCr / YUV**: 
    
    **YCbCr**
    * Componentes: Se compone de tres componentes:
    * Y: Luma o luminancia (brillo, imagen en blanco y negro).
    * Cb: Diferencia de color azul (Blue-difference).
    * Cr: Diferencia de color rojo (Red-difference).

    * Uso: Es el espacio de color estándar para el video digital, como en DVD, Blu-ray, YouTube y otros formatos modernos.separación de luminancia y crominancia — útil en compresión de video.

    **YUV**
    * Componentes: Tradicionalmente se compone de tres componentes:
    * Y: Luminancia.
    * U y V: Señales de crominancia.

    * Uso: Fue desarrollado para la televisión analógica a color, permitiendo la compatibilidad con televisores en blanco y negro.

### 3) Formatos de archivo y compresión

* **BMP**: sin compresión, simple, útil para pruebas.
* **PNG**: compresión sin pérdida (lossless), soporta transparencia (canal alfa).
* **JPEG**: compresión con pérdida (lossy), útil para imágenes fotográficas; cuidado con artefactos al recomprimir.
* **TIFF**: flexible, puede ser lossless o contener múltiples páginas y mayor profundidad de bits.
* **WEBP**: moderno, soporta pérdida y sin pérdida.


### 4) Operaciones básicas y conceptos de procesamiento

* **Operaciones puntuales**: transformación por píxel (p. ej., invertir, escala), rápidas y paralelizables.
* **Operaciones espaciales (convoluciones)**: aplicar kernels / filtros (blur, sharpen, detección de bordes).
* **Operaciones geométricas**: rotación, traslación, escalado y transformaciones afines.
* **Morfología matemática**: erosion, dilatation, apertura y cierre — útiles en limpieza de ruido y post-procesamiento de máscaras.

### 5) Detección de bordes y segmentación básica

* **Filtro Sobel**: gradientes en X e Y — aproximación de derivadas.
* **Filtro Laplacian**: segunda derivada — detecta zonas rápidas de cambio.
* **Canny**: detector de bordes robusto en varios pasos (smoothing, gradiente, non-maximum suppression, hysteresis thresholding).
* **Umbralización (thresholding)**: global, adaptativo y método de Otsu para selección automática de umbral.

### 6) Contornos y análisis geométrico

* Encontrar contornos a partir de una máscara binaria; extraer bounding box, centroides, área y perímetro.
* Uso típico: detección simple de objetos, conteo y medidas geométricas.

---



## Demo guiada con OpenCV (visión general del flujo)

**Objetivo de la demo**: mostrar un pipeline mínimo que carga una imagen, convierte a RGB para visualización en Jupyter, redimensiona, aplica ecualización global y Canny, encuentra contornos y guarda el resultado.

> El código completo y comentado con tipado y pruebas está en la sección "Ejemplo de código".

---

## Código

```main.py```

In [ ]:
"""
Módulo: main.py
Funciones básicas para carga, manipulación y análisis de imágenes con OpenCV.

Buenas prácticas aplicadas:
- Tipado estático (type hints)
- Docstrings con formato estándar
- Uso de logging para trazabilidad
- Manejo de errores robusto
"""

# Permite usar anotaciones de tipo antes de definir funciones o clases (útil para referencias adelantadas)
from __future__ import annotations

# ------------------- Librerías estándar -------------------
import logging  # Permite registrar información, advertencias y errores en lugar de usar print()
from dataclasses import dataclass  # Facilita la creación de clases simples (genera __init__, __repr__, etc.)
from pathlib import Path  # Manejo de rutas multiplataforma (más seguro y flexible que cadenas de texto)
from typing import Optional, Tuple, List  # Tipado estático: mejora documentación y validación en tiempo de desarrollo

# ------------------- Librerías externas -------------------
import cv2  # OpenCV: librería principal para procesamiento de imágenes y visión por computador
import numpy as np  # NumPy: base de datos numérica para manejar arrays multidimensionales (las imágenes son matrices)
import matplotlib.pyplot as plt  # Matplotlib: librería para visualizar imágenes y gráficos

# ------------------- Configuración del logger -------------------
# Configura el sistema de registro para mostrar mensajes informativos y errores en consola
logging.basicConfig(level=logging.INFO, format="%(levelname)s:%(name)s:%(message)s")

# Crea un objeto logger identificado con el nombre "main"
logger = logging.getLogger("main")

# ------------------- Clase de configuración -------------------
@dataclass  # Convierte automáticamente la clase en una estructura de datos con inicializador automático
class ImageProcessingConfig:
    """
    Clase de configuración para operaciones de imagen.

    Attributes:
        interpolation_resize: Tipo de interpolación usada al redimensionar imágenes.
    """
    interpolation_resize: int = cv2.INTER_AREA  # Método de interpolación recomendado para reducir imágenes

# ------------------- Funciones principales -------------------

def load_image(path: str, as_gray: bool = False) -> np.ndarray:
    """Carga una imagen desde `path`."""
    p = Path(path)  # Convierte la ruta recibida en un objeto Path para operaciones seguras de archivos

    if not p.exists():  # Verifica si el archivo realmente existe en el disco
        logger.error("Archivo no encontrado: %s", path)  # Registra el error en los logs
        raise FileNotFoundError(f"Archivo no encontrado: {path}")  # Lanza excepción si no existe

    # Define el modo de lectura dependiendo si se solicita escala de grises o color
    flag = cv2.IMREAD_GRAYSCALE if as_gray else cv2.IMREAD_COLOR

    img = cv2.imread(str(p), flags=flag)  # Carga la imagen usando OpenCV
    if img is None:  # Si OpenCV no puede leer la imagen, devuelve None
        logger.error("cv2.imread devolvió None para %s", path)  # Registra el fallo
        raise IOError(f"No se pudo leer la imagen: {path}")  # Lanza error

    # Muestra en logs información útil de la imagen cargada
    logger.info("Imagen cargada: %s -- shape=%s dtype=%s", path, img.shape, img.dtype)
    return img  # Devuelve la imagen cargada como arreglo NumPy


def save_image(path: str, image: np.ndarray) -> None:
    """Guarda una imagen en disco, creando directorios si es necesario."""
    p = Path(path)  # Convierte la ruta a objeto Path
    p.parent.mkdir(parents=True, exist_ok=True)  # Crea directorios necesarios si no existen

    ok = cv2.imwrite(str(p), image)  # Guarda la imagen en la ruta especificada
    if not ok:  # Si el guardado falla, cv2.imwrite devuelve False
        logger.error("Error guardando la imagen en %s", path)
        raise IOError(f"Error guardando la imagen en {path}")

    logger.info("Imagen guardada en %s", path)  # Registra en logs el éxito del guardado


def to_rgb(image_bgr: np.ndarray) -> np.ndarray:
    """Convierte BGR (formato de OpenCV) a RGB (formato estándar)."""
    if image_bgr.ndim == 2:  # Si la imagen tiene 2 dimensiones, ya está en escala de grises
        return image_bgr  # No se necesita conversión
    return cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)  # Convierte de BGR a RGB


def to_gray(image_bgr: np.ndarray) -> np.ndarray:
    """Convierte una imagen BGR a escala de grises."""
    if image_bgr.ndim == 2:  # Si ya es escala de grises
        return image_bgr
    return cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)  # Usa OpenCV para la conversión


def resize_image(
    image: np.ndarray,
    width: Optional[int] = None,
    height: Optional[int] = None,
    keep_aspect: bool = True,
    config: ImageProcessingConfig = ImageProcessingConfig()
) -> np.ndarray:
    """Redimensiona una imagen conservando la relación de aspecto si se solicita."""
    (h, w) = image.shape[:2]  # Obtiene alto y ancho actuales

    # Si no se especifican dimensiones, se devuelve la imagen original
    if width is None and height is None:
        return image

    # Si se desea mantener la relación de aspecto original
    if keep_aspect:
        if width is None:
            ratio = height / float(h)  # Calcula proporción basada en altura
            dim = (int(w * ratio), height)  # Ajusta ancho proporcionalmente
        elif height is None:
            ratio = width / float(w)
            dim = (width, int(h * ratio))
        else:
            dim = (width, height)
    else:
        # Si no se mantiene la proporción, se usan las dimensiones exactas
        dim = (width if width else w, height if height else h)

    # Redimensiona la imagen con el método de interpolación configurado
    resized = cv2.resize(image, dim, interpolation=config.interpolation_resize)

    # Registra en los logs el cambio de tamaño
    logger.info("resize_image: %s -> %s", (w, h), resized.shape[:2])
    return resized


def rotate_image(image: np.ndarray, angle: float, center: Optional[Tuple[int, int]] = None,
                 scale: float = 1.0) -> np.ndarray:
    """Rota una imagen en torno a un punto dado (por defecto, su centro)."""
    (h, w) = image.shape[:2]  # Obtiene dimensiones de la imagen

    if center is None:  # Si no se especifica el centro de rotación
        center = (w // 2, h // 2)  # Usa el centro geométrico

    # Calcula la matriz de transformación para rotación
    M = cv2.getRotationMatrix2D(center, angle, scale)

    # Aplica la transformación de rotación sobre la imagen
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)

    # Registra el evento de rotación
    logger.info("rotate_image: angle=%s center=%s scale=%s", angle, center, scale)
    return rotated  # Devuelve la imagen rotada


def crop_image(image: np.ndarray, x: int, y: int, w: int, h: int) -> np.ndarray:
    """Recorta una región rectangular (x,y,w,h)."""
    # Recorta los píxeles en el rango indicado y devuelve una copia
    return image[y:y + h, x:x + w].copy()


def histogram_equalization_gray(image_gray: np.ndarray) -> np.ndarray:
    """Equalización de histograma en imágenes en escala de grises usada para mejorar el contraste de una imagen."""
    if image_gray.ndim != 2:  # Valida que sea imagen en gris
        raise ValueError("Debe ser una imagen en escala de grises (2D)")

    # Aplica la equalización con OpenCV
    eq = cv2.equalizeHist(image_gray)

    logger.info("histogram_equalization_gray: completado")  # Log informativo
    return eq  # Devuelve la imagen con histograma equalizado


def apply_clahe_gray(image_gray: np.ndarray, clip_limit: float = 2.0,
                     tile_grid_size: Tuple[int, int] = (8, 8)) -> np.ndarray:
    """Aplica CLAHE: equalización adaptativa del histograma (mejora el contraste local)."""
    # Crea el objeto CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)

    # Aplica el método sobre la imagen
    res = clahe.apply(image_gray)

    # Registra los parámetros utilizados
    logger.info("apply_clahe_gray: clip_limit=%s tile_grid_size=%s", clip_limit, tile_grid_size)
    return res


def gaussian_blur(image: np.ndarray, ksize: Tuple[int, int] = (5, 5)) -> np.ndarray:
    """Aplica un desenfoque Gaussiano (suaviza ruido y bordes)."""
    # Aplica un filtro gaussiano con tamaño de kernel (5x5 por defecto)
    return cv2.GaussianBlur(image, ksize, 0)


def canny_edges(image_gray: np.ndarray, threshold1: float = 100.0, threshold2: float = 200.0) -> np.ndarray:
    """Detecta bordes con el algoritmo de Canny."""
    edges = cv2.Canny(image_gray, threshold1, threshold2)  # Detecta bordes
    logger.info("canny_edges: thresholds=(%s,%s)", threshold1, threshold2)  # Registra umbrales
    return edges  # Devuelve imagen binaria con los bordes detectados

# función crea una máscara binaria que resalta solo los píxeles dentro de un rango específico de color definido en el espacio HSV (Hue, Saturation, Value). muy útil en proyectos de Big Data visual, robótica, seguridad, industria 4.0, agricultura inteligente, etc.

def mask_by_hsv(image_bgr: np.ndarray, lower_hsv: Tuple[int, int, int],
                upper_hsv: Tuple[int, int, int]) -> np.ndarray:
    """Genera una máscara binaria filtrando por rango de color en espacio HSV."""
    hsv = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HSV)  # Convierte de BGR a HSV
    # Crea máscara binaria donde los píxeles dentro del rango son blancos (255)
    mask = cv2.inRange(hsv, np.array(lower_hsv, dtype=np.uint8), np.array(upper_hsv, dtype=np.uint8))
    logger.info("mask_by_hsv: rango=%s-%s", lower_hsv, upper_hsv)
    return mask


def find_and_draw_contours(image_bgr: np.ndarray, binary_mask: np.ndarray,
                           min_area: int = 100) -> Tuple[np.ndarray, List[Tuple[int, int, int, int]]]:
    """Encuentra contornos y dibuja rectángulos alrededor de objetos grandes."""
    # Busca contornos externos en la máscara binaria
    contours, hierarchy = cv2.findContours(binary_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    boxes: List[Tuple[int, int, int, int]] = []  # Lista para almacenar rectángulos detectados
    annotated = image_bgr.copy()  # Copia de la imagen original para dibujar resultados

    for cnt in contours:  # Recorre cada contorno encontrado
        area = cv2.contourArea(cnt)  # Calcula el área del contorno
        if area < min_area:  # Si el área es menor que el mínimo permitido
            continue  # Ignora contornos pequeños
        x, y, w, h = cv2.boundingRect(cnt)  # Calcula el rectángulo delimitador
        boxes.append((x, y, w, h))  # Guarda coordenadas del rectángulo
        cv2.rectangle(annotated, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Dibuja el rectángulo verde

    logger.info("find_and_draw_contours: contornos válidos=%d", len(boxes))  # Log del número de contornos válidos
    return annotated, boxes  # Devuelve la imagen anotada y las coordenadas


def show_image_matplotlib(image: np.ndarray, title: str = "Image") -> None:
    """Muestra una imagen usando Matplotlib (útil en notebooks o Jupyter)."""
    img = to_rgb(image)  # Asegura formato RGB antes de mostrar
    plt.figure(figsize=(8, 6))  # Define tamaño del gráfico
    if img.ndim == 2:  # Si es imagen en gris
        plt.imshow(img, cmap="gray")
    else:
        plt.imshow(img)
    plt.title(title)  # Añade título
    plt.axis("off")  # Oculta ejes
    plt.show()  # Muestra la imagen


# ------------------- Pipeline de ejemplo -------------------
def pipeline_demo(input_path: str, output_path: str) -> None:
    """Pipeline completo: carga, convierte, equaliza, detecta bordes y guarda resultado."""
    img = load_image(input_path, as_gray=False)  # Carga la imagen en color
    gray = to_gray(img)  # Convierte a escala de grises
    eq = apply_clahe_gray(gray)  # Mejora el contraste local
    edges = canny_edges(eq, 50, 150)  # Detecta bordes con Canny

    # Superposición de bordes sobre la imagen original para visualización
    edges_bgr = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)  # Convierte bordes a 3 canales
    overlay = cv2.addWeighted(img, 0.6, edges_bgr, 0.4, 0)  # Combina imagen original y bordes

    # Guarda varias versiones del resultado en disco
    base_name = Path(output_path).stem  # Obtiene nombre base sin extensión
    save_image(f"{base_name}_original.jpg", img)
    save_image(f"{base_name}_bordes.jpg", edges)
    save_image(f"{base_name}_superpuesto.jpg", overlay)
    save_image(output_path, overlay)  # Guarda la versión final

    logger.info("pipeline_demo completado: %s", output_path)  # Informa finalización del proceso


# ------------------- Punto de entrada -------------------
if __name__ == "__main__":  # Solo se ejecuta si el script se corre directamente
    import argparse  # Librería estándar para procesar argumentos de línea de comandos

    # Configura los argumentos esperados
    parser = argparse.ArgumentParser(description="Demo de operaciones básicas con OpenCV")
    parser.add_argument("--input", required=True, help="Ruta de entrada de la imagen")
    parser.add_argument("--output", required=True, help="Ruta de salida de la imagen procesada")
    args = parser.parse_args()  # Parsea los argumentos

    # Ejecuta el pipeline con las rutas dadas por el usuario
    pipeline_demo(args.input, args.output)


> **Notas sobre el código**:
>
> * Use `to_rgb` para visualizar en Jupyter o matplotlib ya que OpenCV usa BGR por defecto.
> * `apply_clahe_gray` es preferible a equalizeHist para evitar sobre-contraste en regiones pequeñas.
> * `find_and_draw_contours` utiliza `RETR_EXTERNAL` para simplificar el ejemplo (solo contornos externos).

---


# ejecutar la app como script

```
python main.py --input sample.jpg --output salida.jpg
```

## Ejercicios prácticos

### Laboratorio 1 — Manipulación básica

**Objetivo**: Ejecutar funciones para cargar, redimensionar, recortar, rotar y guardar imágenes.

**Instrucciones**:

1. Crear un repositorio de práctica con la estructura mínima:

```
cv-lab1/
├── data/
│   └── sample.jpg      # imagen de ejemplo
├── notebooks/
│   └── lab1.ipynb      # notebook con instrucciones y visualizaciones
├── src/
│   └── cv_fundamentals.py  # copiar/usar el módulo de ejemplo
├── requirements.txt
└── README.md
```

2. En `notebooks/lab1.ipynb` realizar los siguientes pasos con código y celdas de explicación:

* Cargar la imagen `data/sample.jpg` y mostrar su shape y dtype.
* Mostrar la imagen original en Jupyter (usar `show_image_matplotlib`).
* Redimensionar la imagen a 800 px de ancho manteniendo aspecto.
* Rotar la imagen 30 grados en sentido antihorario y mostrar el resultado.
* Recortar una región de interés (x=100, y=50, w=200, h=200) y guardar el recorte en `data/crop.jpg`.



---

Repositorio de práctica **cv-lab1** y el notebook **lab1.ipynb** con todo lo solicitado.

Qué incluye 

* `cv-lab1/`

  * `data/sample.jpg` — imagen de ejemplo generada (con una zona roja central, un rectángulo azul y otro verde).
  * `data/crop.jpg` — (se generará al ejecutar el notebook; el notebook guarda el recorte en esa ruta).
  * `notebooks/lab1.ipynb` — notebook con celdas de explicación y todo el código paso a paso: carga, mostrar, redimensionar a 800 px, rotar 30° CCW, recortar ROI (100,50,200,200) y guardar.
  * `src/cv_fundamentals.py` — módulo Python con funciones documentadas y tipadas: `load_image`, `save_image`, `resize_image`, `rotate_image`, `crop_image`, `show_image_matplotlib`. Incluye logging y docstrings siguiendo PEP 8.
  * `requirements.txt` — dependencias sugeridas (OpenCV, numpy, matplotlib, pytest, mypy).
  * `README.md` — instrucciones de uso rápido.

Sugerencias de cómo usarlo

* Abrir `notebooks/lab1.ipynb` con JupyterLab/Jupyter Notebook y ejecutar celda a celda (o `Run → Run All` si ya están instaladas las dependencias).
* Instalar `requirements.txt`.
* Revisar los logs (el módulo usa `logging`).



## Laboratorio 2 — Análisis y segmentación simple

### Objetivo

Aprender a **extraer información útil de imágenes digitales** mediante histogramas, ecualización adaptativa, detección de bordes y segmentación por color, integrando operaciones de preprocesamiento y análisis.

---

### Estructura del repositorio

Se recomienda extender el repositorio anterior (`cv-lab1`) añadiendo un nuevo notebook:

```
cv-lab1/
├── data/
│   └── sample.jpg
├── notebooks/
│   ├── lab1.ipynb
│   └── lab2.ipynb          # ← nuevo notebook con este laboratorio
├── src/
│   └── cv_fundamentals.py
├── requirements.txt
└── README.md
```

---

### Notebook `lab2.ipynb`

#### Paso 1: Histograma de luminancia o escala de grises



In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from src.cv_fundamentals import load_image, show_image_matplotlib

# Cargar imagen en escala de grises
gray = cv2.imread("../data/sample.jpg", cv2.IMREAD_GRAYSCALE)

# Calcular histograma
hist = cv2.calcHist([gray], [0], None, [256], [0, 256])

# Mostrar histograma
plt.figure(figsize=(8, 4))
plt.plot(hist, color='black')
plt.title("Histograma en escala de grises")
plt.xlabel("Intensidad")
plt.ylabel("Frecuencia")
plt.show()


**Explicación**: Un histograma muestra la distribución de intensidades de píxeles. Es fundamental para evaluar contraste.

---

#### Paso 2: Ecualización de histograma adaptativa (CLAHE)



In [ ]:
# Crear objeto CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))

# Aplicar CLAHE
clahe_img = clahe.apply(gray)

# Comparar imágenes
show_image_matplotlib(gray, title="Original (Grises)")
show_image_matplotlib(clahe_img, title="CLAHE")

# Comparar histogramas
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(gray.ravel(), 256, [0, 256], color='gray')
plt.title("Histograma Original")

plt.subplot(1, 2, 2)
plt.hist(clahe_img.ravel(), 256, [0, 256], color='black')
plt.title("Histograma CLAHE")

plt.show()


**Explicación**: CLAHE (Contrast Limited Adaptive Histogram Equalization) mejora el contraste local y evita la sobre-amplificación del ruido.

---

#### Paso 3: Detección de bordes con Canny



In [ ]:
# Bordes con umbral bajo/alto
edges1 = cv2.Canny(gray, 50, 150)
edges2 = cv2.Canny(gray, 100, 200)

# Mostrar comparativa
show_image_matplotlib(edges1, title="Canny 50/150")
show_image_matplotlib(edges2, title="Canny 100/200")


**Explicación**: El algoritmo de Canny busca gradientes de intensidad para detectar bordes. Cambiar los umbrales afecta la sensibilidad.

---

#### Paso 4: Segmentación por color (ejemplo: detectar objeto rojo)



In [ ]:
# Cargar imagen en BGR
image = load_image("../data/sample.jpg")

# Convertir a HSV
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Rango para rojo (dos segmentos debido a circularidad de Hue)
lower_red1 = np.array([0, 100, 100])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([160, 100, 100])
upper_red2 = np.array([179, 255, 255])

# Máscara combinada
mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
mask = cv2.bitwise_or(mask1, mask2)

# Limpiar máscara (morfología)
kernel = np.ones((5, 5), np.uint8)
mask_clean = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
mask_clean = cv2.morphologyEx(mask_clean, cv2.MORPH_CLOSE, kernel)

# Encontrar contornos
contours, _ = cv2.findContours(mask_clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Dibujar bounding boxes
output = image.copy()
for c in contours:
    x, y, w, h = cv2.boundingRect(c)
    cv2.rectangle(output, (x, y), (x+w, y+h), (0, 255, 0), 2)

show_image_matplotlib(output, title="Segmentación por color (Rojo)")


**Explicación**:

* HSV facilita segmentar colores.
* Se aplican operaciones morfológicas (`open` y `close`) para limpiar ruido.
* Los contornos permiten ubicar y medir objetos.

---



## Recursos adicionales y lecturas recomendadas

* Libro clásico: *Digital Image Processing* (Gonzalez & Woods) — teoría y ejemplos.
* Documentación oficial de OpenCV (tutoriales y referencias de funciones).
* Artículos/tutos sobre CLAHE, Canny y transformaciones geométricas.

---


## Extensiones y siguientes clases

* Clase 2: Segmentación avanzada y operaciones morfológicas.
* Clase 3: Detección y reconocimiento de objetos (contornos avanzados, HOG, SIFT/SURF, ORB y un primer acercamiento a redes neuronales convolucionales para visión - transfer learning).

